# Introduction
Here, we'll explore neural networks applied to the fashion-mnist problem.

In [7]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import csv
import pandas as pd

In [8]:
# Functions

def formatArray (dataFrame, columnToExtract) :
    array = dataFrame.values
    target = array[:,columnToExtract]
    params = np.delete(array, columnToExtract, axis = 1)
    return params, target

def loadFashionTrainData():
    return pd.read_csv("fashion-mnist-dataset/fashion-mnist_train.csv")

def loadFashionTestData():
    return pd.read_csv("fashion-mnist-dataset/fashion-mnist_test.csv")

def split_train_test(data, test_ratio):
    np.random.seed(42)
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

# The dataset
First and foremost, we'll open train and test data. The training data is split to obtain validation items and the the target values are also separated from the original data.

In [9]:
fashionTrainDataset = loadFashionTrainData()
fashionTestDataset = loadFashionTestData()
fashionTrain, fashionValidation = split_train_test(fashionTrainDataset, 0.2)
fashionTrain, fashionTarget = formatArray(fashionTrain, 0)

print (fashionTrain[:5])
print (type(fashionTrain))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'numpy.ndarray'>


# Activation and Softmax Functions
First, we'll start by implementing some useful functions seen in class

In [10]:
def step(n):
    if n < 0:
        return 1
    return 0

In [11]:
def sigmoid(n):
    return (1/(1+exp(-n)))

In [12]:
def tanh(n):
    return ((2/(1+exp(-2*n))) - 1)

In [13]:
def relu(n):
    if n < 0:
        return 0
    return n

In [14]:
def leaky_relu(n):
    if x > 0:
        return x
    return 0.01 * x

In [15]:
def softmax(n):
    return np.exp(n)/ np.sum(np.exp(x), axis=0)

## Problem details
- Input dimension: 28x28 = 784 neurons
- Output dimension: 10 classes = 10 neurons

In [ ]:
hidden_layers = 1
input_neurons = 784
hidden_layer_1_neurons = 15
hidden_layer_2_neurons = 15

## Forward Propagation
In this section, we define forward propagation related functions.

In [16]:
def initialize_1hl(w1, w2, b1, b2, hidden_layer_1_neurons, hidden_layer_2_neurons):
    w1 = initialize_weight(w1, hidden_layer_1_neurons)
    w2 = initialize_weight(w2, hidden_layer_2_neurons)
    b1 = initialize_weight(b1, hidden_layer_1_neurons)
    b2 = initialize_weight(b2, hidden_layer_2_neurons)

def initialize_2hl(w1, w2, w3, b1, b2, b3, hidden_layer_1_neurons, hidden_layer_2_neurons, hidden_layer_3_neurons):
    w1 = initialize_weight(w1, hidden_layer_1_neurons)
    w2 = initialize_weight(w2, hidden_layer_2_neurons)
    w3 = initialize_weight(w3, hidden_layer_3_neurons)

    b1 = initialize_weight(b1, hidden_layer_1_neurons)
    b2 = initialize_weight(b2, hidden_layer_2_neurons)
    b3 = initialize_weight(b3, hidden_layer_3_neurons)

def forward_prop_1hl(x, w1, w2, b1, b2):
    y1 = np.dot(x, w1) + b1
    a1 = [relu(n) for n in y1]
    y2 = np.dot(a1, w2) + b2
    return softmax(y2)

def forward_prop_2hl(x, w1, w2, w3, b1, b2, b3):
    y1 = np.dot(x, w1) + b1
    a1 = [relu(n) for n in y1]
    y2 = np.dot(a1, w2) + b2
    a2 = [relu(n) for n in y2]
    y3 = np.dot(a2, w3) + b3
    return softmax(y3)

## Cost function

In [ ]:
# def cost_function(fashionTrainOutput, fashionTarget):
    